<a href="https://colab.research.google.com/github/fertorresfs/Ferramenta-de-Controle-de-Versao-de-Dados-Para-Modelos-de-Machine-Learning/blob/main/ferramenta_para_controle_de_versao_para_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import hashlib
import os
import pandas as pd
import sqlite3
from datetime import datetime

# Diretório para armazenar as versões dos dados
DATA_DIR = "data_versions"
os.makedirs(DATA_DIR, exist_ok=True)

In [ ]:
def gerar_hash(filepath):
    """Gera o hash SHA-256 de um arquivo."""
    hasher = hashlib.sha256()
    with open(filepath, 'rb') as f:
        while True:
            chunk = f.read(4096)
            if not chunk:
                break
            hasher.update(chunk)
    return hasher.hexdigest()

In [ ]:
def versionar_dados(filepath, descricao=""):
    """Versiona um arquivo CSV."""

    # 1. Gerar hash
    file_hash = gerar_hash(filepath)

    # 2. Verificar se a versão já existe
    conn = sqlite3.connect('data_versioning.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM versions WHERE hash=?", (file_hash,))
    existing_version = cursor.fetchone()
    conn.close()

    if existing_version:
        print(f"Versão já existe: {existing_version}")
        return file_hash

    # 3. Copiar o arquivo para o diretório de versões
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    version_filename = f"{timestamp}_{file_hash}.csv"
    version_filepath = os.path.join(DATA_DIR, version_filename)

    df = pd.read_csv(filepath) #lendo com pandas para garantir que é um CSV válido
    df.to_csv(version_filepath, index=False) #copia o arquivo


    # 4. Salvar metadados no banco de dados
    conn = sqlite3.connect('data_versioning.db')
    cursor = conn.cursor()
    cursor.execute("CREATE TABLE IF NOT EXISTS versions (hash TEXT PRIMARY KEY, filepath TEXT, timestamp TEXT, descricao TEXT)")
    cursor.execute("INSERT INTO versions VALUES (?, ?, ?, ?)", (file_hash, version_filepath, timestamp, descricao))
    conn.commit()
    conn.close()

    print(f"Dados versionados com hash: {file_hash}")
    return file_hash

In [ ]:
def carregar_versao(file_hash):
    """Carrega uma versão específica dos dados."""
    conn = sqlite3.connect('data_versioning.db')
    cursor = conn.cursor()
    cursor.execute("SELECT filepath FROM versions WHERE hash=?", (file_hash,))
    result = cursor.fetchone()
    conn.close()

    if result:
        filepath = result[0]
        try:
            df = pd.read_csv(filepath)
            return df
        except FileNotFoundError:
            print(f"Arquivo da versão não encontrado: {filepath}")
            return None
    else:
        print(f"Versão não encontrada para o hash: {file_hash}")
        return None

In [ ]:
# Exemplo de uso:
filepath = "meu_arquivo.csv"
descricao = "Versão inicial dos dados"
file_hash = versionar_dados(filepath, descricao)

In [ ]:
#Para carregar uma versão:
#df_versao = carregar_versao(file_hash)

#if df_versao is not None:
#  print(df_versao.head())